In [23]:
import numpy as np
import random

def rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=False):
    """
    Multiplies two numbers using the RPPM method for a specific bit width.
    A, B: Input integers
    bit_width: Number of bits (8, 16, or 32)
    omit_least_significant: Number of least significant partial products to omit
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val

    # Binary representation
    A_bin = format(A, f'0{bit_width}b')[::-1]  # Reversed binary representation
    B_bin = format(B, f'0{bit_width}b')[::-1]
    partial_products = []

    # Generate partial products
    for i, b in enumerate(B_bin):
        if int(b) == 1:
            partial_product = [(int(a) << i) for a in A_bin]
            partial_products.append(partial_product)

    # Omit least significant partial products
    if len(partial_products) <= omit_least_significant:
        # If all partial products are omitted, return 0
        return 0
    partial_products = partial_products[omit_least_significant:]

    # Compress partial products (ensure it’s not empty)
    if partial_products:
        compressed_sum = np.sum([np.array(pp) for pp in partial_products], axis=0)
    else:
        compressed_sum = [0]

    # Convert back to integer and truncate to bit width
    result = int(''.join(map(str, compressed_sum[::-1])), 2) & max_val

    # Insert Trojan (flip a random bit in the result if flag is set)
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)  # Flip a random bit

    return result


# Approximate Adder Function
def approximate_adder(a, b, bit_width, tolerance=1, insert_trojan=False):
    """
    Performs approximate addition with a tolerance for inaccuracy within a specific bit width.
    a, b: Input integers
    bit_width: Number of bits (8, 16, or 32)
    tolerance: Number of bits where the result may differ
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    a, b = a & max_val, b & max_val

    # Reversed binary representation
    a_bin = format(a, f'0{bit_width}b')[::-1]
    b_bin = format(b, f'0{bit_width}b')[::-1]

    # Perform bitwise addition with approximation
    result = []
    carry = 0
    for i in range(bit_width):
        bit_a = int(a_bin[i]) if i < len(a_bin) else 0
        bit_b = int(b_bin[i]) if i < len(b_bin) else 0

        # Add with carry
        sum_bit = bit_a ^ bit_b ^ carry
        carry = (bit_a & bit_b) | (carry & (bit_a ^ bit_b))

        # Introduce approximation: Ignore carry beyond tolerance
        if i >= tolerance:
            carry = 0

        result.append(sum_bit)

    # Convert result back to integer and truncate to bit width
    result = int(''.join(map(str, result[::-1])), 2) & max_val

    # Insert Trojan (flip a random bit in the result if flag is set)
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)  # Flip a random bit

    return result

# Approximate Adder-based Multiplier
def approximate_adder_multiplier(A, B, bit_width, tolerance=1, insert_trojan=False):
    """
    Multiplies two numbers using an approximate adder for accumulation within a specific bit width.
    A, B: Input integers
    bit_width: Number of bits (8, 16, or 32)
    tolerance: Tolerance for approximation in the adder
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val

    # Generate partial products
    partial_products = []
    for i in range(bit_width):
        if (B >> i) & 1:  # Check if the ith bit of B is 1
            partial_products.append((A << i) & max_val)

    # Approximate addition to accumulate partial products
    result = 0
    for pp in partial_products:
        result = approximate_adder(result, pp, bit_width, tolerance, insert_trojan)

    return result

# Function to compute MAE
def compute_mae(true_result, infected_result, bit_width):
    """
    Calculate the Mean Absolute Error (MAE) between true and infected results.
    true_result: Correct result (non-infected)
    infected_result: Result with Trojan inserted
    bit_width: Number of bits in the result
    """
    # Calculate the absolute error
    error = abs(true_result - infected_result)
    mae = error / (1 << bit_width)  # Normalize by the maximum possible value (2^bit_width)
    return mae

# Test function to run the multipliers and compute MAE
def test_multipliers_and_compute_mae():
    bit_widths = [8, 16, 32]
    A = 45  # Example input A
    B = 20  # Example input B
    insert_trojan = True  # Set to False if you don't want to insert Trojan
    num_iterations = 10  # Number of iterations to run for MAE computation

    for bit_width in bit_widths:
        print(f"\nBit Width: {bit_width}")
        true_rppm_results = []
        infected_rppm_results = []

        true_approx_results = []
        infected_approx_results = []

        # Run the multipliers for multiple iterations
        for _ in range(num_iterations):
            # Compute the correct (non-infected) result
            true_rppm_result = rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=False)
            true_approx_result = approximate_adder_multiplier(A, B, bit_width, tolerance=2, insert_trojan=False)

            # Compute the Trojan-infected result
            infected_rppm_result = rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=True)
            infected_approx_result = approximate_adder_multiplier(A, B, bit_width, tolerance=2, insert_trojan=True)

            # Store results for MAE computation
            true_rppm_results.append(true_rppm_result)
            infected_rppm_results.append(infected_rppm_result)

            true_approx_results.append(true_approx_result)
            infected_approx_results.append(infected_approx_result)

        # Compute MAE for both RPPM and Approximate Adder multipliers
        rppm_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_rppm_results, infected_rppm_results)])
        approx_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_approx_results, infected_approx_results)])

        # Print MAE for both methods
        print(f"RPPM Multiplier MAE (with Trojan): {rppm_mae:.6f}")
        print(f"Approximate Adder Multiplier MAE (with Trojan): {approx_mae:.6f}")

# Run the test to compute MAE
test_multipliers_and_compute_mae()


Bit Width: 8
RPPM Multiplier MAE (with Trojan): 0.000000
Approximate Adder Multiplier MAE (with Trojan): 0.136328

Bit Width: 16
RPPM Multiplier MAE (with Trojan): 0.000000
Approximate Adder Multiplier MAE (with Trojan): 0.090498

Bit Width: 32
RPPM Multiplier MAE (with Trojan): 0.000000
Approximate Adder Multiplier MAE (with Trojan): 0.055119


In [33]:
import numpy as np
import random

def rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=False):
    """
    Multiplies two numbers using the RPPM method for a specific bit width.
    A, B: Input integers
    bit_width: Number of bits (8, 16, or 32)
    omit_least_significant: Number of least significant partial products to omit
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val

    # Binary representation
    A_bin = format(A, f'0{bit_width}b')[::-1]  # Reversed binary representation
    B_bin = format(B, f'0{bit_width}b')[::-1]
    partial_products = []

    # Generate partial products
    for i, b in enumerate(B_bin):
        if int(b) == 1:
            partial_product = [(int(a) << i) for a in A_bin]
            partial_products.append(partial_product)

    # Omit least significant partial products
    if len(partial_products) <= omit_least_significant:
        # If all partial products are omitted, return 0
        return 0
    partial_products = partial_products[omit_least_significant:]

    # Compress partial products bit by bit
    compressed_sum = np.zeros(bit_width, dtype=int)  # Array to hold the compressed sum (bit-wise)

    for pp in partial_products:
        # Add each partial product bit by bit, handling carry properly
        carry = 0
        for i in range(bit_width):
            # Add the partial product bit, carry, and current sum bit
            compressed_sum[i] += pp[i] + carry
            
            # Handle overflow (carry to the next bit) and ensure sum is binary
            if compressed_sum[i] >= 2:  # Binary addition (overflow)
                compressed_sum[i] -= 2  # Keep it binary (0 or 1)
                carry = 1  # Carry for the next bit
            else:
                carry = 0

        # Clip values in compressed_sum to be 0 or 1 (should already be handled by the overflow logic)
        compressed_sum = np.clip(compressed_sum, 0, 1)

    # Convert compressed sum back to integer by joining the bits
    compressed_sum_bits = ''.join(str(bit) for bit in compressed_sum[::-1])  # Reverse bits to match original order
    try:
        result = int(compressed_sum_bits, 2) & max_val
    except ValueError:
        # In case of an invalid bit string, print and return 0 (safety fallback)
        print(f"Invalid compressed sum bits: {compressed_sum_bits}")
        return 0

    # Insert Trojan (flip a random bit in the result if flag is set)
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)  # Flip a random bit

    return result


# Approximate Adder Function
def approximate_adder(a, b, bit_width, tolerance=1, insert_trojan=False):
    """
    Performs approximate addition with a tolerance for inaccuracy within a specific bit width.
    a, b: Input integers
    bit_width: Number of bits (8, 16, or 32)
    tolerance: Number of bits where the result may differ
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    a, b = a & max_val, b & max_val

    # Reversed binary representation
    a_bin = format(a, f'0{bit_width}b')[::-1]
    b_bin = format(b, f'0{bit_width}b')[::-1]

    # Perform bitwise addition with approximation
    result = []
    carry = 0
    for i in range(bit_width):
        bit_a = int(a_bin[i]) if i < len(a_bin) else 0
        bit_b = int(b_bin[i]) if i < len(b_bin) else 0

        # Add with carry
        sum_bit = bit_a ^ bit_b ^ carry
        carry = (bit_a & bit_b) | (carry & (bit_a ^ bit_b))

        # Introduce approximation: Ignore carry beyond tolerance
        if i >= tolerance:
            carry = 0

        result.append(sum_bit)

    # Convert result back to integer and truncate to bit width
    result = int(''.join(map(str, result[::-1])), 2) & max_val

    # Insert Trojan (flip a random bit in the result if flag is set)
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)  # Flip a random bit

    return result

# Approximate Adder-based Multiplier
def approximate_adder_multiplier(A, B, bit_width, tolerance=1, insert_trojan=False):
    """
    Multiplies two numbers using an approximate adder for accumulation within a specific bit width.
    A, B: Input integers
    bit_width: Number of bits (8, 16, or 32)
    tolerance: Tolerance for approximation in the adder
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val

    # Generate partial products
    partial_products = []
    for i in range(bit_width):
        if (B >> i) & 1:  # Check if the ith bit of B is 1
            partial_products.append((A << i) & max_val)

    # Approximate addition to accumulate partial products
    result = 0
    for pp in partial_products:
        result = approximate_adder(result, pp, bit_width, tolerance, insert_trojan)

    return result

# Function to compute MAE
def compute_mae(true_result, infected_result, bit_width):
    """
    Calculate the Mean Absolute Error (MAE) between true and infected results.
    true_result: Correct result (non-infected)
    infected_result: Result with Trojan inserted
    bit_width: Number of bits in the result
    """
    # Calculate the absolute error
    error = abs(true_result - infected_result)
    mae = error / (1 << bit_width)  # Normalize by the maximum possible value (2^bit_width)
    return mae

# Test function to run the multipliers and compute MAE
def test_multipliers_and_compute_mae():
    bit_widths = [8, 16]
    insert_trojan = True  # Set to False if you don't want to insert Trojan
    num_iterations = 100  # Number of iterations to run for MAE computation

    for bit_width in bit_widths:
        print(f"\nBit Width: {bit_width} bits")
        true_rppm_results = []
        infected_rppm_results = []

        true_approx_results = []
        infected_approx_results = []

        # Run the multipliers for multiple iterations
        for _ in range(num_iterations):
            # Generate random A and B for each iteration
            A = random.randint(0, (1 << bit_width) - 1)
            B = random.randint(0, (1 << bit_width) - 1)

            # Compute the correct (non-infected) result
            true_rppm_result = rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=False)
            true_approx_result = approximate_adder_multiplier(A, B, bit_width, tolerance=2, insert_trojan=False)

            # Compute the Trojan-infected result
            infected_rppm_result = rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=True)
            infected_approx_result = approximate_adder_multiplier(A, B, bit_width, tolerance=2, insert_trojan=True)

            # Store results for MAE computation
            true_rppm_results.append(true_rppm_result)
            infected_rppm_results.append(infected_rppm_result)

            true_approx_results.append(true_approx_result)
            infected_approx_results.append(infected_approx_result)

        # Compute MAE for both RPPM and Approximate Adder multipliers
        rppm_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_rppm_results, infected_rppm_results)])
        approx_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_approx_results, infected_approx_results)])

        # Print MAE for both methods in percentage
        print(f"RPPM Multiplier MAE (with Trojan): {rppm_mae * 100:.2f}%")
        print(f"Approximate Adder Multiplier MAE (with Trojan): {approx_mae * 100:.2f}%")

# Run the test to compute MAE
test_multipliers_and_compute_mae()


Bit Width: 8 bits
RPPM Multiplier MAE (with Trojan): 9.00%
Approximate Adder Multiplier MAE (with Trojan): 25.10%

Bit Width: 16 bits
RPPM Multiplier MAE (with Trojan): 6.02%
Approximate Adder Multiplier MAE (with Trojan): 24.27%


In [35]:
import numpy as np
import random

# Existing RPPM and Approximate Adder functions ...

# Narrow-Bit Multiplier (Narrowing)
def narrow_bit_multiplier(A, B, bit_width, narrow_bits=4, insert_trojan=False):
    """
    Multiplies A and B, but narrows down the bit-width of partial products.
    `narrow_bits`: Number of bits retained in the partial products.
    """
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val
    result = 0

    for i in range(bit_width):
        if (B >> i) & 1:
            # Perform the multiplication and narrow the bit-width of the partial product
            partial_product = (A << i) & max_val
            result += (partial_product >> (bit_width - narrow_bits))  # Narrowing to `narrow_bits`
    
    result = result & max_val  # Ensure result fits within bit width

    # Insert Trojan if specified
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)

    return result

# Rounding-Based Multiplier (Round to nearest power of 2)
def rounding_multiplier(A, B, bit_width, insert_trojan=False):
    """
    Multiplies A and B, rounds the result to the nearest multiple of 2^bit_width.
    """
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val
    result = A * B

    # Round the result to the nearest multiple of 2^bit_width
    rounding_factor = 1 << bit_width
    result = (result + (rounding_factor // 2)) // rounding_factor  # Rounding to nearest
    
    # Ensure the result is within bit-width
    result = result & max_val

    # Insert Trojan if specified
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)

    return result

# Truncation-Based Multiplier (Truncate extra bits)
def truncation_multiplier(A, B, bit_width, insert_trojan=False):
    """
    Multiplies A and B, then truncates the least significant bits beyond `bit_width`.
    """
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val
    result = A * B

    # Truncate the result by masking off the extra bits
    result = result & max_val

    # Insert Trojan if specified
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)

    return result

# Function to compute MAE
def compute_mae(true_result, infected_result, bit_width):
    """
    Calculate the Mean Absolute Error (MAE) between true and infected results.
    true_result: Correct result (non-infected)
    infected_result: Result with Trojan inserted
    bit_width: Number of bits in the result
    """
    # Calculate the absolute error
    error = abs(true_result - infected_result)
    mae = error / (1 << bit_width)  # Normalize by the maximum possible value (2^bit_width)
    return mae

# Test function to run the multipliers and compute MAE
def test_multipliers_and_compute_mae():
    bit_widths = [8, 16]  # You can add 32-bit if needed
    insert_trojan = True  # Set to False if you don't want to insert Trojan
    num_iterations = 100  # Number of iterations to run for MAE computation

    for bit_width in bit_widths:
        print(f"\nBit Width: {bit_width} bits")
        true_rppm_results = []
        infected_rppm_results = []

        true_approx_results = []
        infected_approx_results = []

        true_narrow_results = []
        infected_narrow_results = []

        true_rounding_results = []
        infected_rounding_results = []

        true_trunc_results = []
        infected_trunc_results = []

        # Run the multipliers for multiple iterations
        for _ in range(num_iterations):
            # Generate random A and B for each iteration
            A = random.randint(0, (1 << bit_width) - 1)
            B = random.randint(0, (1 << bit_width) - 1)

            # Compute the correct (non-infected) result
            true_rppm_result = rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=False)
            true_approx_result = approximate_adder_multiplier(A, B, bit_width, tolerance=2, insert_trojan=False)
            true_narrow_result = narrow_bit_multiplier(A, B, bit_width, narrow_bits=4, insert_trojan=False)
            true_rounding_result = rounding_multiplier(A, B, bit_width, insert_trojan=False)
            true_trunc_result = truncation_multiplier(A, B, bit_width, insert_trojan=False)

            # Compute the Trojan-infected result
            infected_rppm_result = rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=True)
            infected_approx_result = approximate_adder_multiplier(A, B, bit_width, tolerance=2, insert_trojan=True)
            infected_narrow_result = narrow_bit_multiplier(A, B, bit_width, narrow_bits=4, insert_trojan=True)
            infected_rounding_result = rounding_multiplier(A, B, bit_width, insert_trojan=True)
            infected_trunc_result = truncation_multiplier(A, B, bit_width, insert_trojan=True)

            # Store results for MAE computation
            true_rppm_results.append(true_rppm_result)
            infected_rppm_results.append(infected_rppm_result)

            true_approx_results.append(true_approx_result)
            infected_approx_results.append(infected_approx_result)

            true_narrow_results.append(true_narrow_result)
            infected_narrow_results.append(infected_narrow_result)

            true_rounding_results.append(true_rounding_result)
            infected_rounding_results.append(infected_rounding_result)

            true_trunc_results.append(true_trunc_result)
            infected_trunc_results.append(infected_trunc_result)

        # Compute MAE for all methods
        rppm_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_rppm_results, infected_rppm_results)])
        approx_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_approx_results, infected_approx_results)])
        narrow_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_narrow_results, infected_narrow_results)])
        rounding_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_rounding_results, infected_rounding_results)])
        trunc_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_trunc_results, infected_trunc_results)])

        # Print MAE for each method in percentage
        print(f"RPPM Multiplier MAE (with Trojan): {rppm_mae * 100:.2f}%")
        print(f"Approximate Adder Multiplier MAE (with Trojan): {approx_mae * 100:.2f}%")
        print(f"Narrow-Bit Multiplier MAE (with Trojan): {narrow_mae * 100:.2f}%")
        print(f"Rounding Multiplier MAE (with Trojan): {rounding_mae * 100:.2f}%")
        print(f"Truncation Multiplier MAE (with Trojan): {trunc_mae * 100:.2f}%")

# Run the test to compute MAE
test_multipliers_and_compute_mae()


Bit Width: 8 bits
RPPM Multiplier MAE (with Trojan): 10.46%
Approximate Adder Multiplier MAE (with Trojan): 23.33%
Narrow-Bit Multiplier MAE (with Trojan): 11.95%
Rounding Multiplier MAE (with Trojan): 11.20%
Truncation Multiplier MAE (with Trojan): 10.13%

Bit Width: 16 bits
RPPM Multiplier MAE (with Trojan): 6.72%
Approximate Adder Multiplier MAE (with Trojan): 24.12%
Narrow-Bit Multiplier MAE (with Trojan): 5.68%
Rounding Multiplier MAE (with Trojan): 7.60%
Truncation Multiplier MAE (with Trojan): 8.24%


In [37]:
import numpy as np
import random

def rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=False):
    """
    Multiplies two numbers using the RPPM method for a specific bit width.
    A, B: Input integers
    bit_width: Number of bits (8, 16, or 32)
    omit_least_significant: Number of least significant partial products to omit
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val

    # Binary representation
    A_bin = format(A, f'0{bit_width}b')[::-1]  # Reversed binary representation
    B_bin = format(B, f'0{bit_width}b')[::-1]
    partial_products = []

    # Generate partial products
    for i, b in enumerate(B_bin):
        if int(b) == 1:
            partial_product = [(int(a) << i) for a in A_bin]
            partial_products.append(partial_product)

    # Omit least significant partial products
    if len(partial_products) <= omit_least_significant:
        # If all partial products are omitted, return 0
        return 0
    partial_products = partial_products[omit_least_significant:]

    # Compress partial products bit by bit
    compressed_sum = np.zeros(bit_width, dtype=int)  # Array to hold the compressed sum (bit-wise)

    for pp in partial_products:
        # Add each partial product bit by bit, handling carry properly
        carry = 0
        for i in range(bit_width):
            # Add the partial product bit, carry, and current sum bit
            compressed_sum[i] += pp[i] + carry
            
            # Handle overflow (carry to the next bit) and ensure sum is binary
            if compressed_sum[i] >= 2:  # Binary addition (overflow)
                compressed_sum[i] -= 2  # Keep it binary (0 or 1)
                carry = 1  # Carry for the next bit
            else:
                carry = 0

        # Clip values in compressed_sum to be 0 or 1 (should already be handled by the overflow logic)
        compressed_sum = np.clip(compressed_sum, 0, 1)

    # Convert compressed sum back to integer by joining the bits
    compressed_sum_bits = ''.join(str(bit) for bit in compressed_sum[::-1])  # Reverse bits to match original order
    try:
        result = int(compressed_sum_bits, 2) & max_val
    except ValueError:
        # In case of an invalid bit string, print and return 0 (safety fallback)
        print(f"Invalid compressed sum bits: {compressed_sum_bits}")
        return 0

    # Insert Trojan (flip a random bit in the result if flag is set)
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)  # Flip a random bit

    return result


# Approximate Adder Function
def approximate_adder(a, b, bit_width, tolerance=1, insert_trojan=False):
    """
    Performs approximate addition with a tolerance for inaccuracy within a specific bit width.
    a, b: Input integers
    bit_width: Number of bits (8, 16, or 32)
    tolerance: Number of bits where the result may differ
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    a, b = a & max_val, b & max_val

    # Reversed binary representation
    a_bin = format(a, f'0{bit_width}b')[::-1]
    b_bin = format(b, f'0{bit_width}b')[::-1]

    # Perform bitwise addition with approximation
    result = []
    carry = 0
    for i in range(bit_width):
        bit_a = int(a_bin[i]) if i < len(a_bin) else 0
        bit_b = int(b_bin[i]) if i < len(b_bin) else 0

        # Add with carry
        sum_bit = bit_a ^ bit_b ^ carry
        carry = (bit_a & bit_b) | (carry & (bit_a ^ bit_b))

        # Introduce approximation: Ignore carry beyond tolerance
        if i >= tolerance:
            carry = 0

        result.append(sum_bit)

    # Convert result back to integer and truncate to bit width
    result = int(''.join(map(str, result[::-1])), 2) & max_val

    # Insert Trojan (flip a random bit in the result if flag is set)
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)  # Flip a random bit

    return result

# Approximate Adder-based Multiplier
def approximate_adder_multiplier(A, B, bit_width, tolerance=1, insert_trojan=False):
    """
    Multiplies two numbers using an approximate adder for accumulation within a specific bit width.
    A, B: Input integers
    bit_width: Number of bits (8, 16, or 32)
    tolerance: Tolerance for approximation in the adder
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val

    # Generate partial products
    partial_products = []
    for i in range(bit_width):
        if (B >> i) & 1:  # Check if the ith bit of B is 1
            partial_products.append((A << i) & max_val)

    # Approximate addition to accumulate partial products
    result = 0
    for pp in partial_products:
        result = approximate_adder(result, pp, bit_width, tolerance, insert_trojan)

    return result

# Narrow-Bit Multiplier (Narrowing)
def narrow_bit_multiplier(A, B, bit_width, narrow_bits=4, insert_trojan=False):
    """
    Multiplies A and B, but narrows down the bit-width of partial products.
    `narrow_bits`: Number of bits retained in the partial products.
    """
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val
    result = 0

    for i in range(bit_width):
        if (B >> i) & 1:
            # Perform the multiplication and narrow the bit-width of the partial product
            partial_product = (A << i) & max_val
            result += (partial_product >> (bit_width - narrow_bits))  # Narrowing to `narrow_bits`
    
    result = result & max_val  # Ensure result fits within bit width

    # Insert Trojan if specified
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)

    return result

# Rounding-Based Multiplier (Round to nearest power of 2)
def rounding_multiplier(A, B, bit_width, insert_trojan=False):
    """
    Multiplies A and B, rounds the result to the nearest multiple of 2^bit_width.
    """
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val
    result = A * B

    # Round the result to the nearest multiple of 2^bit_width
    rounding_factor = 1 << bit_width
    result = (result + (rounding_factor // 2)) // rounding_factor  # Rounding to nearest
    
    # Ensure the result is within bit-width
    result = result & max_val

    # Insert Trojan if specified
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)

    return result

# Truncation-Based Multiplier (Truncate extra bits)
def truncation_multiplier(A, B, bit_width, insert_trojan=False):
    """
    Multiplies A and B, then truncates the least significant bits beyond `bit_width`.
    """
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val
    result = A * B

    # Truncate the result by masking off the extra bits
    result = result & max_val

    # Insert Trojan if specified
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)

    return result

# Function to compute MAE
def compute_mae(true_result, infected_result, bit_width):
    """
    Calculate the Mean Absolute Error (MAE) between true and infected results.
    true_result: Correct result (non-infected)
    infected_result: Result with Trojan inserted
    bit_width: Number of bits in the result
    """
    # Calculate the absolute error
    error = abs(true_result - infected_result)
    mae = (error / (1 << bit_width)) * 100  # Normalize by the maximum possible value (2^bit_width) and convert to percentage
    return mae

# Test function to run the multipliers and compute MAE
def test_multipliers_and_compute_mae():
    bit_widths = [8, 16]  # Can add 32 if needed
    insert_trojan = True  # Set to False if you don't want to insert Trojan
    num_iterations = 100  # Number of iterations to run for MAE computation

    for bit_width in bit_widths:
        print(f"\nBit Width: {bit_width} bits")
        true_rppm_results = []
        infected_rppm_results = []

        true_approx_results = []
        infected_approx_results = []

        true_narrow_results = []
        infected_narrow_results = []

        true_rounding_results = []
        infected_rounding_results = []

        true_truncation_results = []
        infected_truncation_results = []

        # Run the multipliers for multiple iterations
        for _ in range(num_iterations):
            # Generate random A and B for each iteration
            A = random.randint(0, (1 << bit_width) - 1)
            B = random.randint(0, (1 << bit_width) - 1)

            # Compute the correct (non-infected) result
            true_rppm_result = rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=False)
            true_approx_result = approximate_adder_multiplier(A, B, bit_width, tolerance=2, insert_trojan=False)
            true_narrow_result = narrow_bit_multiplier(A, B, bit_width, narrow_bits=4, insert_trojan=False)
            true_rounding_result = rounding_multiplier(A, B, bit_width, insert_trojan=False)
            true_truncation_result = truncation_multiplier(A, B, bit_width, insert_trojan=False)

            # Compute the Trojan-infected result
            infected_rppm_result = rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=True)
            infected_approx_result = approximate_adder_multiplier(A, B, bit_width, tolerance=2, insert_trojan=True)
            infected_narrow_result = narrow_bit_multiplier(A, B, bit_width, narrow_bits=4, insert_trojan=True)
            infected_rounding_result = rounding_multiplier(A, B, bit_width, insert_trojan=True)
            infected_truncation_result = truncation_multiplier(A, B, bit_width, insert_trojan=True)

            # Store results for MAE computation
            true_rppm_results.append(true_rppm_result)
            infected_rppm_results.append(infected_rppm_result)

            true_approx_results.append(true_approx_result)
            infected_approx_results.append(infected_approx_result)

            true_narrow_results.append(true_narrow_result)
            infected_narrow_results.append(infected_narrow_result)

            true_rounding_results.append(true_rounding_result)
            infected_rounding_results.append(infected_rounding_result)

            true_truncation_results.append(true_truncation_result)
            infected_truncation_results.append(infected_truncation_result)

        # Compute MAE for each method
        rppm_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_rppm_results, infected_rppm_results)])
        approx_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_approx_results, infected_approx_results)])
        narrow_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_narrow_results, infected_narrow_results)])
        rounding_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_rounding_results, infected_rounding_results)])
        truncation_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_truncation_results, infected_truncation_results)])

        # Print MAE for all methods
        print(f"RPPM Multiplier MAE (with Trojan): {rppm_mae:.2f}%")
        print(f"Approximate Adder Multiplier MAE (with Trojan): {approx_mae:.2f}%")
        print(f"Narrow-Bit Multiplier MAE (with Trojan): {narrow_mae:.2f}%")
        print(f"Rounding Multiplier MAE (with Trojan): {rounding_mae:.2f}%")
        print(f"Truncation Multiplier MAE (with Trojan): {truncation_mae:.2f}%")

# Run the test to compute MAE
test_multipliers_and_compute_mae()


Bit Width: 8 bits
RPPM Multiplier MAE (with Trojan): 9.00%
Approximate Adder Multiplier MAE (with Trojan): 22.45%
Narrow-Bit Multiplier MAE (with Trojan): 10.51%
Rounding Multiplier MAE (with Trojan): 11.02%
Truncation Multiplier MAE (with Trojan): 10.01%

Bit Width: 16 bits
RPPM Multiplier MAE (with Trojan): 6.03%
Approximate Adder Multiplier MAE (with Trojan): 24.93%
Narrow-Bit Multiplier MAE (with Trojan): 6.58%
Rounding Multiplier MAE (with Trojan): 7.42%
Truncation Multiplier MAE (with Trojan): 6.78%


In [39]:
import numpy as np
import random

def rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=False):
    """
    Multiplies two numbers using the RPPM method for a specific bit width.
    A, B: Input integers
    bit_width: Number of bits (8, 16, or 32)
    omit_least_significant: Number of least significant partial products to omit
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val

    # Binary representation
    A_bin = format(A, f'0{bit_width}b')[::-1]  # Reversed binary representation
    B_bin = format(B, f'0{bit_width}b')[::-1]
    partial_products = []

    # Generate partial products
    for i, b in enumerate(B_bin):
        if int(b) == 1:
            partial_product = [(int(a) << i) for a in A_bin]
            partial_products.append(partial_product)

    # Omit least significant partial products
    if len(partial_products) <= omit_least_significant:
        # If all partial products are omitted, return 0
        return 0
    partial_products = partial_products[omit_least_significant:]

    # Compress partial products bit by bit
    compressed_sum = np.zeros(bit_width, dtype=int)  # Array to hold the compressed sum (bit-wise)

    for pp in partial_products:
        # Add each partial product bit by bit, handling carry properly
        carry = 0
        for i in range(bit_width):
            # Add the partial product bit, carry, and current sum bit
            compressed_sum[i] += pp[i] + carry
            
            # Handle overflow (carry to the next bit) and ensure sum is binary
            if compressed_sum[i] >= 2:  # Binary addition (overflow)
                compressed_sum[i] -= 2  # Keep it binary (0 or 1)
                carry = 1  # Carry for the next bit
            else:
                carry = 0

        # Clip values in compressed_sum to be 0 or 1 (should already be handled by the overflow logic)
        compressed_sum = np.clip(compressed_sum, 0, 1)

    # Convert compressed sum back to integer by joining the bits
    compressed_sum_bits = ''.join(str(bit) for bit in compressed_sum[::-1])  # Reverse bits to match original order
    try:
        result = int(compressed_sum_bits, 2) & max_val
    except ValueError:
        # In case of an invalid bit string, print and return 0 (safety fallback)
        print(f"Invalid compressed sum bits: {compressed_sum_bits}")
        return 0

    # Insert Trojan (flip a random bit in the result if flag is set)
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)  # Flip a random bit

    return result


# Approximate Adder Function
def approximate_adder(a, b, bit_width, tolerance=1, insert_trojan=False):
    """
    Performs approximate addition with a tolerance for inaccuracy within a specific bit width.
    a, b: Input integers
    bit_width: Number of bits (8, 16, or 32)
    tolerance: Number of bits where the result may differ
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    a, b = a & max_val, b & max_val

    # Reversed binary representation
    a_bin = format(a, f'0{bit_width}b')[::-1]
    b_bin = format(b, f'0{bit_width}b')[::-1]

    # Perform bitwise addition with approximation
    result = []
    carry = 0
    for i in range(bit_width):
        bit_a = int(a_bin[i]) if i < len(a_bin) else 0
        bit_b = int(b_bin[i]) if i < len(b_bin) else 0

        # Add with carry
        sum_bit = bit_a ^ bit_b ^ carry
        carry = (bit_a & bit_b) | (carry & (bit_a ^ bit_b))

        # Introduce approximation: Ignore carry beyond tolerance
        if i >= tolerance:
            carry = 0

        result.append(sum_bit)

    # Convert result back to integer and truncate to bit width
    result = int(''.join(map(str, result[::-1])), 2) & max_val

    # Insert Trojan (flip a random bit in the result if flag is set)
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)  # Flip a random bit

    return result

# Approximate Adder-based Multiplier
def approximate_adder_multiplier(A, B, bit_width, tolerance=1, insert_trojan=False):
    """
    Multiplies two numbers using an approximate adder for accumulation within a specific bit width.
    A, B: Input integers
    bit_width: Number of bits (8, 16, or 32)
    tolerance: Tolerance for approximation in the adder
    insert_trojan: Boolean flag to insert a Trojan by flipping a random bit in the result
    """
    # Ensure inputs are within bit width
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val

    # Generate partial products
    partial_products = []
    for i in range(bit_width):
        if (B >> i) & 1:  # Check if the ith bit of B is 1
            partial_products.append((A << i) & max_val)

    # Approximate addition to accumulate partial products
    result = 0
    for pp in partial_products:
        result = approximate_adder(result, pp, bit_width, tolerance, insert_trojan)

    return result

# Combined Rounding and Truncation Multiplier
def rounding_truncation_multiplier(A, B, bit_width, rounding=True, insert_trojan=False):
    """
    Multiplies A and B, then either rounds or truncates the result based on the `rounding` flag.
    """
    max_val = (1 << bit_width) - 1
    A, B = A & max_val, B & max_val
    result = A * B

    if rounding:
        # Round the result to the nearest multiple of 2^bit_width
        rounding_factor = 1 << bit_width
        result = (result + (rounding_factor // 2)) // rounding_factor  # Round to nearest
    else:
        # Truncate to fit within the desired bit-width
        result = result & max_val

    # Insert Trojan if specified
    if insert_trojan:
        trojan_bit = random.randint(0, bit_width - 1)
        result ^= (1 << trojan_bit)

    return result

# Function to compute MAE
def compute_mae(true_result, infected_result, bit_width):
    """
    Calculate the Mean Absolute Error (MAE) between true and infected results.
    true_result: Correct result (non-infected)
    infected_result: Result with Trojan inserted
    bit_width: Number of bits in the result
    """
    error = abs(true_result - infected_result)
    mae = (error / (1 << bit_width)) * 100  # Normalize by the maximum possible value and convert to percentage
    return mae

# Test function to run the multipliers and compute MAE
def test_multipliers_and_compute_mae():
    bit_widths = [8, 16]  # Bit widths to test
    insert_trojan = True  # Flag to insert Trojan
    num_iterations = 100  # Number of iterations to run for MAE computation

    for bit_width in bit_widths:
        print(f"\nBit Width: {bit_width} bits")
        true_rppm_results = []
        infected_rppm_results = []
        true_approx_results = []
        infected_approx_results = []
        true_round_trunc_results = []
        infected_round_trunc_results = []

        # Run the multipliers for multiple iterations
        for _ in range(num_iterations):
            # Generate random A and B for each iteration
            A = random.randint(0, (1 << bit_width) - 1)
            B = random.randint(0, (1 << bit_width) - 1)

            # Compute the correct (non-infected) result
            true_rppm_result = rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=False)
            true_approx_result = approximate_adder_multiplier(A, B, bit_width, tolerance=2, insert_trojan=False)
            true_round_trunc_result = rounding_truncation_multiplier(A, B, bit_width, rounding=True, insert_trojan=False)

            # Compute the Trojan-infected result
            infected_rppm_result = rppm_multiplier(A, B, bit_width, omit_least_significant=2, insert_trojan=True)
            infected_approx_result = approximate_adder_multiplier(A, B, bit_width, tolerance=2, insert_trojan=True)
            infected_round_trunc_result = rounding_truncation_multiplier(A, B, bit_width, rounding=True, insert_trojan=True)

            # Store results for MAE computation
            true_rppm_results.append(true_rppm_result)
            infected_rppm_results.append(infected_rppm_result)
            true_approx_results.append(true_approx_result)
            infected_approx_results.append(infected_approx_result)
            true_round_trunc_results.append(true_round_trunc_result)
            infected_round_trunc_results.append(infected_round_trunc_result)

        # Compute MAE for each method
        rppm_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_rppm_results, infected_rppm_results)])
        approx_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_approx_results, infected_approx_results)])
        round_trunc_mae = np.mean([compute_mae(tr, ir, bit_width) for tr, ir in zip(true_round_trunc_results, infected_round_trunc_results)])

        # Print MAE for all methods
        print(f"RPPM Multiplier MAE (with Trojan): {rppm_mae:.2f}%")
        print(f"Approximate Adder Multiplier MAE (with Trojan): {approx_mae:.2f}%")
        print(f"Rounding/Truncation Multiplier MAE (with Trojan): {round_trunc_mae:.2f}%")

# Run the test to compute MAE
test_multipliers_and_compute_mae()


Bit Width: 8 bits
RPPM Multiplier MAE (with Trojan): 10.65%
Approximate Adder Multiplier MAE (with Trojan): 23.67%
Rounding/Truncation Multiplier MAE (with Trojan): 14.57%

Bit Width: 16 bits
RPPM Multiplier MAE (with Trojan): 4.37%
Approximate Adder Multiplier MAE (with Trojan): 25.44%
Rounding/Truncation Multiplier MAE (with Trojan): 6.43%
